# How to use the Large Language Models BERT and RoBERTa

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook explains how you can use a Large Language Model (LLM) throught the Transformer package. LLMs models are published regularly on the huggingface platform: https://huggingface.co/models

These models are very big (Gigabytes) and require a computer with sufficient memory to load. Furthermore, loading these models takes some time as well. It is also possible to copy such a model to your disk and to load the local copy. Still a substantial memory is needed to load it.

There is whole family of transformer models developed by different research groups and published on the Huggingface platform. We will look at two popular LLMs BERT and its sequel RoBERTa, specifically its crosslingual variant XML-RoBERTa.

BERT (Bidirectional Encoder Representations from Transformers) is a transformers model pretrained on a large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, without human labelling. BERT was pretrained with two objectives:

* Masked language modeling (MLM): taking a sentence, the model randomly masks 15% of the words in the input. It next runs the entire masked sentence through the model to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which only mask the future/next tokens. BERT allows the model to learn a bidirectional representation of the sentence.
* Next sentence prediction (NSP): the model concatenates two masked sentences as input during pretraining. Sometimes they correspond to sentences that were following each other in the original text (positive example), sometimes it is a random sentence (negative example). The model then has to predict if the two sentences were following each other or not.

The core trick of the model is **Attention**, hence the seminal paper "Attention is all you need" (Vaswani et al 2017). Attention refers to the principle that the embedding representation of a word or token in a sequence is determined for the purpose of e.g. predicting the masked tokens but with the help of the other tokens. Likewise, the model learns which words in the context pay most attention to the role of predicting masked words. This is applied to all words in a sequence and over many (11 to more than 20) layers with different attention heads.

Eventually running many times over large amounts of data, the models learn an inner representation of the English language (a language model) that can then be used to represent sentences in texts as contextual vectors in downstream tasks, instead of feature engineered vectors. If you have a dataset of labeled sentences for instance, you can train a standard classifier using the representations produced by the BERT model as inputs. 

![finetune](fine-tuning.png)

Whereas terrabytes of text data has been used for pretraining the LLM, relatively little data is needed to fine-tine the model for a specific classification task. A small training set (Megabytes) is represented using the LLM and a classification head is trained to associate labels, such as sentiment.

RoBERTa is a more efficient sequel to BERT in which the next sentence prediction was dropped, more data, more layers and wider contexts were used (Liu et al 2019). It often gives better performance than BERT in downstream tasks but it is bigger and more heavy to use.

XLM-RoBERTa is a multilingual version of RoBERTa. It is pre-trained on 2.5TB of filtered CommonCrawl data containing 100 languages (Lample and Conneau 2019). XLM-RoBERTa can represent text in any of these languages and it can be finetuned in one language (or more) and applied to all these languages. This makes it possible to profit from the English training data that is available for all languages captured by the model even if there is no labeled data in that specific language.


## References
Vaswani, Ashish, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin. "Attention is all you need." Advances in neural information processing systems 30 (2017).https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

Alexis Conneau, Kartikay Khandelwal, Naman Goyal, Vishrav Chaudhary, Guillaume Wenzek, Francisco Guzma ́n, Edouard Grave, Myle Ott, Luke Zettle- moyer, and Veselin Stoyanov. 2020. Unsupervised Cross-lingual Representation Learning at Scale. In Proceedings of the 58th Annual Meeting of the Asso- ciation for Computational Linguistics, pages 8440– 8451, Online. Association for Computational Lin- guistics.

Devlin, Jacob, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).

Liu, Yinhan, Myle Ott, Naman Goyal, Jingfei Du, Mandar Joshi, Danqi Chen, Omer Levy, Mike Lewis, Luke Zettlemoyer, and Veselin Stoyanov. "Roberta: A robustly optimized bert pretraining approach." arXiv preprint arXiv:1907.11692 (2019).


## Loading a model in a prefabbed NLP pipeline

We will start with the English case-sensitive BERT model that is provided by the Huggingface platform. It is possible to load the model itself in combination with its tokenizer to get a representation of a text for all words/tokens across all layers. It is however rather complex to exploit these representations for specific tasks. Huggingface therefore provides an option to create a pipeline to perform an NLP task with a pretrained model:

    "The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering."

More information can be found here: https://huggingface.co/transformers/v3.0.2/main_classes/pipelines.html

You can find information on the different tasks and how to call these. We will use these pipelines in this class for a gentle introduction. In notebook 4.3, will use the pipeline module to load fine-tuned models to perform sentiment analysis and emotion detection. In notebook 4.4, we try out finetuned crosslingual models.

This notebook requires installing the deep learning packages **pytorch** and **transformers**. For this lab you should use the transformers version 4.16.0. Once installed, you can comment out the next cell.

In [2]:
#!pip install torch==2.0.0
#!pip install transformers==4.16.0

Below, we use the two pretrained Language Models BERT and XLM-RoBERTa to perform the basic task of predicting a masked word in the context of a sentence. 

The models are available on the huggingface.co platform:

* https://huggingface.co/bert-base-cased
* https://huggingface.co/xlm-roberta-base

On the platform you can find documentation, the actual files that make up the model (in Files and versions) and an online demo: Hosted inference API. Look around a bit in the platform to see what is there and how the models are released. For fun, download BERT's ```vocab.txt``` file and inspect it to see what word pieces BERT represents.

You can try out the model online, but we want to use it in our programs locally. Therefore we are going to load it in this notebook from huggingface.co and use it in a ```pipeline``` to do the MASKED-task: predicting the worlds that are most likely to occur in a specific masked position in a sentence. For this, we use the **fill-mask** pipeline as an interface for the model.

We create an instance of the pipeline class, where we specify the task **fill-mask** and the name of the model. When creating the instance, the constructor scans the Huggingface platform for the model (or your local cache) and its configuration for the task. If you have loaded the model before, it will find it cached on your local disk. If not, it will download it from Huggingface, which may take some time.

In [1]:
from transformers import pipeline

pipe = pipeline('fill-mask', model='bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


When downloading the model and creating the pipeline, you may see a warning that some weights (parameters) are not downloaded, specifically for the ```[CLS]``` token. The ```[CLS]``` token is used for text classification and is not needed for masked token prediction. By creating a pipeline for masked-token-prediction, the downloader inferred it can do without these weights to make it more efficient.

Once our ```pipe``` has been created, we can use it by giving it the text as input but replace the words we want to predict with ```[MASK]```: 

In [2]:
for res in pipe('A keyboard and a [MASK] are connected to the computer.'):
    print(res['sequence'], res['score'])

A keyboard and a mouse are connected to the computer. 0.9537658095359802
A keyboard and a keyboard are connected to the computer. 0.005814594682306051
A keyboard and a screen are connected to the computer. 0.005326828453689814
A keyboard and a monitor are connected to the computer. 0.0022222495172172785
A keyboard and a microphone are connected to the computer. 0.0020763936918228865


In [17]:
for res in pipe('The cat chased the [MASK] for minutes.'):
    print(res['sequence'], res['score'])

The cat chased the dog for minutes. 0.11454426497220993
The cat chased the mouse for minutes. 0.06594355404376984
The cat chased the cat for minutes. 0.04837101697921753
The cat chased the bird for minutes. 0.042223118245601654
The cat chased the rabbit for minutes. 0.03711289167404175


We can see from the different predictions for the MASKED position that the context triggered different words. We also see that the probabilities descrease rapidly for the results.

Instead of words, we can also prompt for names:

In [18]:
for res in pipe('Mr [MASK] was charged with murder.'):
    print(res['sequence'], res['score'])

Mr Smith was charged with murder. 0.018348269164562225
Mr Brown was charged with murder. 0.010945719666779041
Mr Jones was charged with murder. 0.00945380050688982
Mr Johnson was charged with murder. 0.009399165399372578
Mr Williams was charged with murder. 0.00915615539997816


And the other way around, we can prompt for what people are charged for...

In [20]:
for res in pipe('Mr Williams was charged with [MASK].'):
    print(res['sequence'], res['score'])
    
for res in pipe('Mrs Williams was charged with [MASK].'):
    print(res['sequence'], res['score'])

Mr Williams was charged with murder. 0.2875919044017792
Mr Williams was charged with assault. 0.28199896216392517
Mr Williams was charged with theft. 0.08729249984025955
Mr Williams was charged with fraud. 0.04526205733418465
Mr Williams was charged with corruption. 0.02898520603775978
Mrs Williams was charged with murder. 0.3436300754547119
Mrs Williams was charged with assault. 0.2426556646823883
Mrs Williams was charged with theft. 0.07686605304479599
Mrs Williams was charged with rape. 0.04139390215277672
Mrs Williams was charged with fraud. 0.03615877404808998


Think about what these models do when they generate these answers! Did they index knowledge on individuals as facts or do they only have a hunch about what vocabulary items can be expected in this position?

Also note that by only changing ```Mr``` into ```Mrs``` we get a different result and different scores. The differences does not make much sense to me as it would be more likely that a ```Mr Williams``` is charged with rape than a ```Mrs Williams```. 

This tells us that 1) models are biased for gender and 2) they do not use common sense reasoning and knowledge as we do.

Next, we will load XLM-RoBERTa from huggingface in a fill-mask pipeline as we have done before for English BERT to perform word predictions in different languages covered by the model.

In [21]:
from transformers import pipeline

xlmpipe = pipeline('fill-mask', model='xlm-roberta-base')

Once loaded, we can use this ```xlmpipe``` in a similar way. The only difference being that the masked-token in this model is represented as ```<mask```:

In [22]:
for res in xlmpipe('I wish I was a <mask>.'):
    print(res['sequence'], res['score'])

I wish I was a girl. 0.09122451394796371
I wish I was a teenager. 0.07413558661937714
I wish I was a poet. 0.06350739300251007
I wish I was a child. 0.0401216596364975
I wish I was a virgin. 0.0334009975194931


So let us now try some texts in other languages with the same XLM model:

In [23]:
for res in xlmpipe('Ik wou dat ik een <mask> was.'):
    print(res['sequence'], res['score'])

Ik wou dat ik een vrouw was. 0.12419804185628891
Ik wou dat ik een meisje was. 0.05872365087270737
Ik wou dat ik een engel was. 0.04070601984858513
Ik wou dat ik een homo was. 0.03964477777481079
Ik wou dat ik een mens was. 0.039212655276060104


In [24]:
for res in xlmpipe('Ich mochte gerne ein <mask> sein.'):
    print(res['sequence'], res['score'])

Ich mochte gerne ein Mädchen sein. 0.05431733280420303
Ich mochte gerne ein Moderator sein. 0.042532797902822495
Ich mochte gerne ein Mensch sein. 0.04099571332335472
Ich mochte gerne ein Paar sein. 0.029234454035758972
Ich mochte gerne ein Engel sein. 0.028842106461524963


In [25]:
for res in xlmpipe('The <mask> was charged with murder.'):
    print(res['sequence'], res['score'])

The man was charged with murder. 0.35578620433807373
The suspect was charged with murder. 0.1935453712940216
The woman was charged with murder. 0.08891762793064117
The victim was charged with murder. 0.08566372096538544
The officer was charged with murder. 0.03933117538690567


In [26]:
for res in xlmpipe('De <mask> is aangeklaagd voor moord.'):
    print(res['sequence'], res['score'])

De verdachte is aangeklaagd voor moord. 0.4496532678604126
De man is aangeklaagd voor moord. 0.32273969054222107
De vrouw is aangeklaagd voor moord. 0.04201311990618706
De politie is aangeklaagd voor moord. 0.019671106711030006
De bestuurder is aangeklaagd voor moord. 0.01778404600918293


So note that the returned results are different across the languages. This means there are different associations triggered even though the semantics of the input sentences is the same. So in addition to biases such as gender, race, etc. there are also linguistic-cultural biases in the models.

You can check which languages are covered in XLM-RoBERTa and try out other word predictions in other languages yourself.

BERT and RoBERTa use a different representation for the masked token. This is an incidental difference. The can check the vocabulary file of the model to see what tokens are used.

You can find the actual files on huggingface.co. Go to a model of your choice and use the option "Files and versions" to get a listing of the files for a model. The file ```tokenizer.json``` contains the vocabulary:

* https://huggingface.co/bert-base-cased/raw/main/tokenizer.json
* https://huggingface.co/xlm-roberta-base/blob/main/tokenizer.json

These files a big but can be opened in a browser as raw data or downloaded individually.

## End of notebook